In [42]:
np.arange(1, 101 + 1, 5)

array([  1,   6,  11,  16,  21,  26,  31,  36,  41,  46,  51,  56,  61,
        66,  71,  76,  81,  86,  91,  96, 101])

In [1]:
import os
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

import torch

import neurox.interpretation.utils as utils
import neurox.interpretation.ablation as ablation
import neurox.interpretation.linear_probe as linear_probe

from ttsxai.utils.utils import read_ljs_metadata


log_dir = '/nas/users/dahye/kw/tts/ttsxai/logs/probe_tacotron2_duration'
data_activation_dir = "/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow"


In [75]:
a = get_neuron_ordering_for_linear_regression(probe)

In [74]:
def get_random_neurons(probe, probability):
    """
    Get random neurons from a trained probe.

    This method returns a random set of neurons based on the probability. Each
    neuron is either discarded or included based on a uniform random variable's
    value (included if its less than probability, discarded otherwise)

    Parameters
    ----------
    probe : interpretation.linear_probe.LinearProbe
        Trained probe model
    probability : float
        Real number between 0 and 1, with 0 representing no selection and 1
        representing selection of all neurons.

    Returns
    -------
    random_neurons : numpy.ndarray
        Numpy array with random neurons

    """
    weights = list(probe.parameters())[0].data.cpu()
    weights = np.abs(weights.numpy())

    mask = np.random.random((weights.shape[1],))
    idx = np.where(mask <= probability)[0]

    return idx

get_random_neurons(probe, 0.1).shape

(212,)

In [35]:
data_dict_train = np.load(os.path.join(log_dir, 'data', 'train_data.npz'), allow_pickle=True)
X_train = data_dict_train['X']
y_train = data_dict_train['y']

data_dict_test = np.load(os.path.join(log_dir, 'data', 'test_data.npz'), allow_pickle=True)
X_test = data_dict_test['X']
y_test = data_dict_test['y']

# ignore zero duration
X_train = X_train[y_train > 0]
y_train = y_train[y_train > 0]
X_test = X_test[y_test > 0]
y_test = y_test[y_test > 0]

# we predict log duration
y_train = np.log(y_train)
y_test = np.log(y_test)

probe = linear_probe.LinearProbe(X_train.shape[1], 1)

In [124]:
def get_neuron_ordering_for_linear_regression(probe, search_stride=100):
    """
    Get neuron ordering for a specific class from a trained probe.

    Parameters
    ----------
    probe : interpretation.linear_probe.LinearProbe
        Trained probe model
    search_stride : int, optional
        Number of steps to divide the weight mass percentage

    Returns
    -------
    neuron_ordering : numpy.ndarray
        Array of neurons ordered by their importance for the specified class
    """
    # class_idx = class_to_idx[class_name]
    weights = list(probe.parameters())[0].data.cpu().numpy()
    abs_weights = np.abs(weights[0])
    si = np.argsort(abs_weights)[::-1]
    print(si[:5])
    print(si[-5:])
    print(abs_weights[si])

    neuron_orderings = []
    for p in range(1, search_stride + 1):
        percentage = p / search_stride
        total_mass = np.sum(abs_weights)
        sorted_idx = np.argsort(abs_weights)[::-1]  # Sort in descending order
        cum_sums = np.cumsum(abs_weights[sorted_idx])
        selected_neurons = sorted_idx[cum_sums <= total_mass * percentage]
        neuron_orderings.extend(selected_neurons)

    # Remove duplicates while preserving order
    neuron_ordering = list(dict.fromkeys(neuron_orderings))

    return np.array(neuron_ordering)


In [125]:
get_neuron_ordering_for_linear_regression(probe)

[1851  911  923  778 1865]
[1456   99   59  590  707]
[2.2092573e-02 2.2089176e-02 2.2084175e-02 ... 1.8686858e-05 1.7975630e-05
 1.0325978e-05]


array([1851,  911,  923, ...,   59,  590,  707])

In [28]:
data_dict = dict(np.load(os.path.join(log_dir, 'scores', 'mlpprobe.npz')))
data_dict['test']

array([0.07462117, 0.07352313, 0.07349195], dtype=float32)

In [20]:
data_dict = dict(np.load(os.path.join(log_dir, 'scores', 'probe_noreg.npz')))
data_dict['test']

FileNotFoundError: [Errno 2] No such file or directory: '/nas/users/dahye/kw/tts/ttsxai/logs/probe_tacotron2_duration/scores/probe_noreg.npz'

In [43]:
for k in range(4):
    data_dict = dict(np.load(os.path.join(log_dir, 'scores', f'probe_layer_{k}_noreg.npz')))
    print(data_dict['test'][-1])

0.121908344
0.09826692
0.0920727
0.084278084


In [33]:
for k in range(4):
    data_dict = dict(np.load(os.path.join(log_dir, 'scores', f'probe_layer_{k}.npz')))
    print(data_dict['test'][-1])

0.1406279
0.11491856
0.0981596
0.08764746


In [ ]:
1	0.31382546
6	0.28406215
11	0.18350205
16	0.18289219
21	0.17996849
26	0.16872445
31	0.1597819
36	0.15191793
41	0.13707356
46	0.13129628
51	0.12141481
56	0.1132296
61	0.10900084
66	0.10591225
71	0.10438707
76	0.101809256
81	0.099342935

In [ ]:
1
6
11
16
21
26
31
36
41
46
51
56
61
66
71
76
81


In [ ]:
0.3218547
0.3184304
0.27302012
0.2604244
0.25735116
0.24943794
0.24705687
0.24389279
0.243521
0.24152999
0.23963873
0.23363096
0.22532807
0.21038997
0.20773752
0.20760679
0.19826505
0.19652925
0.18913865
0.1773968
0.1771514

In [99]:
ordering = get_neuron_ordering_for_linear_regression(probe)

In [101]:
ordering

array([1851,  911,  923, ...,   59,  590,  707])

In [102]:
orderling = ordering[::-1]

In [106]:
orderling

array([ 707,  590,   59, ...,  923,  911, 1851])

In [140]:
for k in range(1, 101+1, 5):
    data_dict = dict(np.load(os.path.join(log_dir, 'scores', f'probe_selected_bottom_{k}_noreg.npz')))
    # print(k)#, data_dict['test'][-1], sep='\t')
    print(data_dict['test'][-1])
    # print('\n')

0.3218547
0.3184304
0.27302012
0.2604244
0.25735116
0.24943794
0.24705687
0.24389279
0.243521
0.24152999
0.23963873
0.23363096
0.22532807
0.21038997
0.20773752
0.20760679
0.19826505
0.19652925
0.18913865
0.1773968
0.1771514


In [131]:
for k in range(1, 101+1, 5):
    data_dict = dict(np.load(os.path.join(log_dir, 'scores', f'probe_selected_{k}.npz')))
    # print(k)#, data_dict['test'][-1], sep='\t')
    print(data_dict['test'][-1])
    # print('\n')

0.31382546
0.28406215
0.18350205
0.18289219
0.17996849
0.16872445
0.1597819
0.15191793
0.13707356
0.13129628
0.12141481
0.1132296
0.10900084
0.10591225
0.10438707
0.101809256
0.099342935
0.09711998
0.09277097
0.090178065
0.08916971


In [94]:
for k in range(1, 101+1, 5):
    data_dict = dict(np.load(os.path.join(log_dir, 'scores', f'probe_selected_{k}_noreg.npz')))
    # print(k)#, data_dict['test'][-1], sep='\t')
    print(data_dict['test'][-1])
    # print('\n')

0.32171428
0.31003788
0.28170297
0.27537477
0.27299696
0.24196056
0.23885222
0.22891366
0.21511868
0.21135941
0.20518248
0.1943123
0.18957333
0.18737522
0.18533146


FileNotFoundError: [Errno 2] No such file or directory: '/nas/users/dahye/kw/tts/ttsxai/logs/probe_tacotron2_duration/scores/probe_selected_76_noreg.npz'

In [ ]:
    # load pre-trained probe
    probe = linear_probe.train_logistic_regression_probe(
        X, y, lambda_l1=0.001, lambda_l2=0.001,
        num_epochs=0)
    probe.load_state_dict(
        torch.load(os.path.join(log_dir, 'models', 'probe.pth')))

In [2]:
# data_dict_train = np.load(os.path.join(log_dir, 'data', 'train_data.npz'), allow_pickle=True)
# X_train = data_dict_train['X']
# y_train = data_dict_train['y']

data_dict_test = np.load(os.path.join(log_dir, 'data', 'test_data.npz'), allow_pickle=True)
X_test = data_dict_test['X']
y_test = data_dict_test['y']

# ignore zero duration
# X_train = X_train[y_train > 0]
# y_train = y_train[y_train > 0]
X_test = X_test[y_test > 0]
y_test = y_test[y_test > 0]

# we predict log duration
# y_train = np.log(y_train)
y_test = np.log(y_test)

# probe = linear_probe.LinearProbe(X_train.shape[1], 1)

In [3]:
probe = linear_probe.LinearProbe(X_test.shape[1], 1)

In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

from neurox.interpretation.linear_probe import l1_penalty, l2_penalty, LinearProbe

In [5]:
X_train=X_test
y_train=y_test
task_type="regression"
lambda_l1=0
lambda_l2=0
num_epochs=1
batch_size=64
learning_rate=0.001
# probe=None

progressbar = utils.get_progress_bar()
print("Training %s probe" % (task_type))
# Check if we can use GPU's for training
use_gpu = torch.cuda.is_available()

if lambda_l1 is None or lambda_l2 is None:
    raise ValueError("Regularization weights cannot be None")

print("Creating model...")
if task_type == "classification":
    num_classes = len(set(y_train))
    if num_classes <= 1:
        raise ValueError(
            "Classification problem must have more than one target class"
        )
else:
    num_classes = 1
print("Number of training instances:", X_train.shape[0])
if task_type == "classification":
    print("Number of classes:", num_classes)

if probe is None:
    probe = LinearProbe(X_train.shape[1], num_classes)
if use_gpu:
    probe = probe.cuda()

if task_type == "classification":
    criterion = nn.CrossEntropyLoss()
elif task_type == "regression":
    criterion = nn.MSELoss()
else:
    raise ValueError("Invalid `task_type`")

optimizer = torch.optim.Adam(probe.parameters(), lr=learning_rate)

X_tensor = torch.from_numpy(X_train)
y_tensor = torch.from_numpy(y_train)

for epoch in range(num_epochs):
    num_tokens = 0
    avg_loss = 0
    for inputs, labels in progressbar(
        utils.batch_generator(X_tensor, y_tensor, batch_size=batch_size),
        desc="epoch [%d/%d]" % (epoch + 1, num_epochs),
    ):
        num_tokens += inputs.shape[0]
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
        inputs = inputs.float()
        inputs = Variable(inputs)
        labels = Variable(labels)

        # Forward + Backward + Optimize
        # optimizer.zero_grad()

        outputs = probe(inputs)
        if task_type == "regression":
            outputs = outputs.squeeze()
        weights = list(probe.parameters())[0]
        loss = (
            criterion(outputs, labels)
            + lambda_l1 * l1_penalty(weights)
            + lambda_l2 * l2_penalty(weights)
        )
        # loss.backward()
        # optimizer.step()

        avg_loss += loss.item()
        print(loss.item(), num_tokens)
        # break

    print(
        "Epoch: [%d/%d], Loss: %.4f"
        % (epoch + 1, num_epochs, avg_loss / num_tokens)
    )

Training regression probe
Creating model...
Number of training instances: 33942


/home/dahye/anaconda3/envs/ttsxai/lib/python3.7/site-packages/ipykernel_launcher.py:54: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch [1/1]: 0it [00:00, ?it/s]

3.4089431762695312 64
3.653441905975342 128
3.6800765991210938 192
3.7993295192718506 256
3.760315418243408 320
3.669862985610962 384
3.481282949447632 448
4.594367027282715 512
3.500903606414795 576
3.8371059894561768 640
3.9819250106811523 704
3.6700704097747803 768
4.37565279006958 832
4.1844801902771 896
4.0132293701171875 960
3.435898542404175 1024
3.790529251098633 1088
3.8998231887817383 1152
3.6397061347961426 1216
3.6958298683166504 1280
4.028511047363281 1344
4.026883125305176 1408
4.601248741149902 1472
4.196177959442139 1536
3.6182403564453125 1600
4.037708282470703 1664
4.045098304748535 1728
4.20054292678833 1792
3.93361496925354 1856
3.776142120361328 1920
4.2568769454956055 1984
3.8073465824127197 2048
3.6250782012939453 2112
4.056900978088379 2176
4.106441497802734 2240
3.3177528381347656 2304
4.61070442199707 2368
3.7675700187683105 2432
4.101977348327637 2496
3.6550137996673584 2560
3.709392547607422 2624
3.440361976623535 2688
3.899653911590576 2752
3.87792086601257

In [8]:
scores_test = linear_probe.evaluate_probe(probe, X_test, y_test, metric='mse')
print( '[* test score]', scores_test)

Evaluating: 0it [00:00, ?it/s]

Score (mse) of the probe: 3.77
[* test score] {'__OVERALL__': 3.7677808}


In [9]:
scores_test

{'__OVERALL__': 3.7677808}

In [15]:
0.1154 * 32

3.6928

In [7]:
import os
import numpy as np
from tqdm import tqdm

from ttsxai.utils.utils import read_ljs_metadata

In [17]:
data_activation_dir = "/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow"

# Dictionary keys to filter
keys_to_filter = read_ljs_metadata(mode='test')

# List to store filtered paths
npz_files = []

# Iterate over all files in the directory
for file in os.listdir(data_activation_dir):
    # Check only for .npz files
    if file.endswith('.npz'):
        # Extract the identifier part from the file name (e.g., 'LJ037-0213')
        identifier = file.split('.')[0]

        # If this identifier is included in the dictionary keys, add to the list
        if identifier in keys_to_filter:
            full_path = os.path.join(data_activation_dir, file)
            npz_files.append(full_path)

In [21]:
all_phonesymbols = []
all_articulatory_features = []
all_activations = []
for file in tqdm(npz_files):
    data_dict = np.load(file, allow_pickle=True)
    activations = data_dict['activations'].item()
    
    phonesymbols = data_dict['phonesymbols']
    articulatory_features = data_dict['articulatory_features']

    all_phonesymbols.append(phonesymbols)
    all_articulatory_features.append(articulatory_features)
    all_activations.append(activations)

100%|██████████| 500/500 [00:01<00:00, 314.62it/s]


In [38]:
def create_tensors(
    source_tokens,
    target_tokens,
    activations,
    task_specific_tag="Unknown",
    task_type="classification",
    binarized_tag=None,
    balance_data=False,
    dtype=None,
    ignore_tags=None
):
    """
    Method to pre-process loaded datasets into tensors that can be used to train
    probes and perform analyis on. The input tokens are represented as list of
    sentences, where each sentence is a list of tokens. Each token also has
    an associated label. All tokens from all sentences are flattened into one
    dimension in the returned tensors. The returned tensors will thus have
    ``total_num_tokens`` rows.

    Parameters
    ----------
    source_tokens: list of arrays
        List of sentences, where each sentence is a list of tokens. 
        e.g., [array(['M', 'ER0', 'IY1', ...], array(['JH', 'AH1', 'S', ...])
    target_tokens: list of arrays
        It depends on task_type. For example, in classification task, 
        articulatory_features can be used as targets.
        e.g., [['Bilabial', 'Vowel', 'Vowel', ...], [...]]
    activations : list of dictionary
        e.g., activations[0] has
        {'conv_0': ..., 'conv_1': ...}
        where each key has *sentence representations*, where each *sentence representation*
        is a numpy matrix of shape.
    task_specific_tag : str
        Label to assign tokens with unseen labels. This is particularly useful
        if some labels are never seen during train, but are present in the dev
        or test set. This is usually set to the majority class in the task.
    task_type : str
        Either "classification" or "regression", indicate the kind of task that
        is being probed.
    binarized_tag : str, optional
            Tag/Label to create binary data. All other labels in the dataset are changed
            to OTHER. Defaults to None in which case the data labels are processed as-is.
    balance_data : bool, optional
        Whether the incoming data should be balanced. Data is balanced using
        utils.balance_binary_class_data for binary data and utils.balance_multi_class_data
        for multi-class data using undersampling. Defaults to False.
    dtype : str, optional
        None if the dtype of the activation tensor should be the same dtype as in the activations input
        e.g. 'float16' or 'float32' to enforce half-precision or full-precision floats

    """
    assert (
        task_type == "classification" or task_type == "regression"
    ), "Invalid model type"

    if ignore_tags is None:
        ignore_tags = []

    filtered_source_tokens = []
    filtered_target_tokens = []

    for source_sentence, target_sentence in zip(source_tokens, target_tokens):
        filtered_source_sentence = []
        filtered_target_sentence = []
        for source_token, target_token in zip(source_sentence, target_sentence):
            if target_token not in ignore_tags:
                filtered_source_sentence.append(source_token)
                filtered_target_sentence.append(target_token)
        filtered_source_tokens.append(np.array(filtered_source_sentence))
        filtered_target_tokens.append(np.array(filtered_target_sentence))

    source_tokens = filtered_source_tokens
    target_tokens = filtered_target_tokens

    num_tokens = count_tokens(source_tokens)
    print("Number of tokens: ", num_tokens)

    # Concatenate activations
    concatenated_activations = [np.concatenate(list(d.values()), axis=1) for d in activations]
    num_neurons = concatenated_activations[0].shape[1]

    if task_type == "classification":
        if binarized_tag:
            label2idx = {binarized_tag: 1, "OTHER": 0}
            idx2label = {1: binarized_tag, 0: "OTHER"}
        else:
            label2idx = tok2idx(target_tokens)
            idx2label = idx2tok(label2idx)

    src2idx = tok2idx(source_tokens)
    idx2src = idx2tok(src2idx)
    nidx2name = neuronidx2name(activations[0])

    print("length of source dictionary: ", len(src2idx))
    if task_type == "classification":
        print("length of target dictionary: ", len(label2idx))

    if dtype == None:
        dtype = concatenated_activations[0].dtype
    X = np.zeros((num_tokens, num_neurons), dtype=dtype)
    if task_type == "classification":
        y = np.zeros((num_tokens,), dtype=np.int)
    else:
        y = np.zeros((num_tokens,), dtype=np.float32)

    example_set = set()

    idx = 0
    for instance_idx, instance in enumerate(target_tokens):
        for token_idx, _ in enumerate(instance):
            if idx < num_tokens:
                X[idx] = concatenated_activations[instance_idx][token_idx, :]

            example_set.add(source_tokens[instance_idx][token_idx])
            if task_type == "classification":
                current_target_token = target_tokens[instance_idx][token_idx]
                if binarized_tag and current_target_token != binarized_tag:
                    current_target_token = "OTHER"
                y[idx] = label2idx[current_target_token]
            elif task_type == "regression":
                y[idx] = float(target_tokens[instance_idx][token_idx])

            idx += 1

    print(idx)
    print("Total instances: %d" % (num_tokens))
    # print(list(example_set)[:20])
    print(list(example_set))

    print("Number of samples: ", X.shape[0])

    # if balance_data:
    #     print("Balancing data ... ")
    #     if binarized_tag:
    #         X, y = balance_binary_class_data(X, y)
    #     else:
    #         X, y = balance_multi_class_data(X, y)
    #     print("Number of samples after balancing: ", X.shape[0])

    labels, freqs = np.unique(y, return_counts=True)

    print("Stats: Labels with their frequencies in the final set")
    for idx, label in enumerate(labels):
        print(idx2label[label], freqs[idx])

    if task_type == "classification":
        return X, y, (label2idx, idx2label, src2idx, idx2src, nidx2name)
    return X, y, (src2idx, idx2src)


def count_tokens(source):
    """
    Utility function to count the total number of tokens in a dataset.
    """
    return sum([len(t) for t in source])


def tok2idx(tokens):
    """
    Utility function to generate unique indices for a set of tokens.
    """
    uniq_tokens = set().union(*tokens)
    return {p: idx for idx, p in enumerate(uniq_tokens)}


def idx2tok(srcidx):
    """
    Utility function to an inverse mapping from a ``tok2idx`` mapping.
    """
    return {v: k for k, v in srcidx.items()}


# Create the mapping for a single activation dictionary
def neuronidx2name(d):
    mapping = {}
    current_start_index = 0
    for layer_name, activations in d.items():
        # Calculate the end index for this activation
        end_index = current_start_index + activations.shape[1] - 1
        for i in range(current_start_index, end_index + 1):
            mapping[i] = f'{layer_name}__{i - current_start_index}'
        current_start_index = end_index + 1
    return mapping


X, y, mapping = create_tensors(
    all_phonesymbols,
    all_articulatory_features,
    all_activations,
    task_type='classification',
    # ignore_tags=['Space', 'Punctuation']
)

Number of tokens:  43250
length of source dictionary:  78
length of target dictionary:  12


/home/dahye/anaconda3/envs/ttsxai/lib/python3.7/site-packages/ipykernel_launcher.py:103: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


43250
Total instances: 43250
['ZH', 'ER2', 'NG', 'P', 'IY2', 'AA2', 'OY2', 'UH2', 'AW1', 'EH1', 'AO1', 'W', 'EY1', 'UH0', 'AO0', 'Y', 'AY2', 'N', 'AA0', 'UW2', 'S', "'", 'Z', 'CH', 'UW0', 'TH', ' ', 'OW1', 'OY1', 'IH2', 'AA1', 'UH1', 'SH', '(', 'IY1', 'AW2', 'ER1', 'G', 'D', 'UW1', ')', 'AE0', 'OW0', 'OW2', '.', 'IH0', 'EY0', ':', 'IH1', 'V', 'F', '-', ',', 'AH2', 'AE2', ';', 'AE1', 'EH0', 'AY0', 'AY1', 'JH', 'EH2', 'AH0', 'T', 'IY0', 'L', 'B', 'R', 'AO2', 'EY2', 'AW0', 'DH', 'HH', 'M', 'K', 'AH1', 'ER0', '?']
Number of samples:  43250
Stats: Labels with their frequencies in the final set
Dental 1302
Labio-Velar 790
Palatal 184
Labiodental 1411
Vowel 13305
Punctuation 1016
Alveolar 12338
Post-Alveolar 660
Bilabial 2225
Velar 1503
Space 7989
Glottal 527


In [30]:
mapping

({'Dental': 0,
  'Labio-Velar': 1,
  'Palatal': 2,
  'Labiodental': 3,
  'Vowel': 4,
  'Punctuation': 5,
  'Alveolar': 6,
  'Post-Alveolar': 7,
  'Bilabial': 8,
  'Velar': 9,
  'Space': 10,
  'Glottal': 11},
 {0: 'Dental',
  1: 'Labio-Velar',
  2: 'Palatal',
  3: 'Labiodental',
  4: 'Vowel',
  5: 'Punctuation',
  6: 'Alveolar',
  7: 'Post-Alveolar',
  8: 'Bilabial',
  9: 'Velar',
  10: 'Space',
  11: 'Glottal'},
 {'ZH': 0,
  'ER2': 1,
  'NG': 2,
  'P': 3,
  'IY2': 4,
  'AA2': 5,
  'OY2': 6,
  'UH2': 7,
  'AW1': 8,
  'EH1': 9,
  'AO1': 10,
  'W': 11,
  'EY1': 12,
  'UH0': 13,
  'AO0': 14,
  'Y': 15,
  'AY2': 16,
  'N': 17,
  'AA0': 18,
  'UW2': 19,
  'S': 20,
  "'": 21,
  'Z': 22,
  'CH': 23,
  'UW0': 24,
  'TH': 25,
  ' ': 26,
  'OW1': 27,
  'OY1': 28,
  'IH2': 29,
  'AA1': 30,
  'UH1': 31,
  'SH': 32,
  '(': 33,
  'IY1': 34,
  'AW2': 35,
  'ER1': 36,
  'G': 37,
  'D': 38,
  'UW1': 39,
  ')': 40,
  'AE0': 41,
  'OW0': 42,
  'OW2': 43,
  '.': 44,
  'IH0': 45,
  'EY0': 46,
  ':': 47,
  '

In [22]:
all_phonesymbols

[array(['DH', 'AH0', ' ', 'F', 'AO1', 'R', 'M', 'Z', ' ', 'AH1', 'V', ' ',
        'P', 'R', 'IH1', 'N', 'T', 'AH0', 'D', ' ', 'L', 'EH1', 'T', 'ER0',
        'Z', ' ', 'SH', 'UH1', 'D', ' ', 'B', 'IY1', ' ', 'B', 'Y', 'UW1',
        'T', 'AH0', 'F', 'AH0', 'L', ',', ' ', 'AH0', 'N', 'D', ' ', 'DH',
        'AE1', 'T', ' ', 'DH', 'EH1', 'R', ' ', 'ER0', 'EY1', 'N', 'JH',
        'M', 'AH0', 'N', 'T', ' ', 'AA1', 'N', ' ', 'DH', 'AH0', ' ', 'P',
        'EY1', 'JH', ' ', 'SH', 'UH1', 'D', ' ', 'B', 'IY1', ' ', 'R',
        'IY1', 'Z', 'AH0', 'N', 'AH0', 'B', 'AH0', 'L', ' ', 'AH0', 'N',
        'D', ' ', 'EY1', ' ', 'HH', 'EH1', 'L', 'P', ' ', 'T', 'UW1', ' ',
        'DH', 'AH0', ' ', 'SH', 'EY1', 'P', 'AH0', 'L', 'IH2', 'N', 'Z',
        ' ', 'AH1', 'V', ' ', 'DH', 'AH0', ' ', 'L', 'EH1', 'T', 'ER0',
        'Z', ' ', 'DH', 'EH0', 'M', 'S', 'EH1', 'L', 'V', 'Z', '.'],
       dtype='<U3'),
 array(['IH1', 'T', ' ', 'W', 'AA1', 'Z', ' ', 'S', 'EH1', 'D', ' ', 'IH0',
        'N', ' ', 'L'

In [16]:
len(npz_files)

12500